In a first step we have classified all knots with at most 15 crossings that share the same 0-surgeries. 

Now we want to find actually the pair of knots (K,K') that share the same 0-surgery and realize the minimum complexity c(K)+c(K').

In a previous step we have created a list for every prime knot K with at most 9 crossings that contains all prime knots with at most 19 crossings with the same Alexander polynomial and complexity at most 25. These are candidates that might share the same 0-surgery. We continue to work with this list.

First we load the necessary code:

In [1]:
import snappy
import regina
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def change_notation(dt_code):
    """
    Changes Dowker-Thistlewait notation from alphabetical to numerical
    Input:
        dt_code (string): alphabetical DT notation
    Return:
        (string): numerical DT notation
    """
    alpha = "abcdefghijklmnopqrstuvwxyz"
    Alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = []
    for letter in dt_code:
        if letter in alpha:
            result.append(2* (alpha.index(letter) + 1))
        elif letter in Alpha:
            result.append(-2 * (Alpha.index(letter) + 1))
        else:
            print(dt_code)
    return "DT: " + str([tuple(result)])

def fill_triangulation(M):
    '''
    Fills all cusps but one.
    '''
    if M.num_cusps()==1:
        return M
    M=M.filled_triangulation([0])
    M=fill_triangulation(M)
    return M

#### This is Dunfield's util.py from his exceptional census

####  for a snappy manifold M descibed as a single filling of a cusp (so do filled_triangulation() as needed) 
####  the command regina_name(M) gives what regina identifies M as

"""

This file provides functions for working with Regina (with a little
help from SnapPy) to:

1. Give a standard name ("identify") manifolds, especially Seifert and
   graph manifolds.

2. Find essential tori.

3. Try to compute the JSJ decomposition.

"""

import regina
import snappy
import re
import networkx as nx

def appears_hyperbolic(M):
    acceptable = ['all tetrahedra positively oriented',
                  'contains negatively oriented tetrahedra']
    return M.solution_type() in acceptable and M.volume() > 0

def children(packet):
    child = packet.firstChild()
    while child:
        yield child
        child = child.nextSibling()

def to_regina(data):
    if hasattr(data, '_to_string'):
        data = data._to_string()
    if isinstance(data, str):
        if data.find('(') > -1:
            data = closed_isosigs(data)[0]
        return regina.Triangulation3(data)
    assert isinstance(data, regina.Triangulation3)
    return data

def extract_vector(surface):
    """
    Extract the raw vector of the (almost) normal surface in Regina's
    NS_STANDARD coordinate system.
    """
    S = surface
    T = S.triangulation()
    n = T.countTetrahedra()
    ans = []
    for i in range(n):
        for j in range(4):
            ans.append(S.triangles(i, j))
        for j in range(3):
            ans.append(S.quads(i, j))
    A = regina.NormalSurface(T, regina.NS_STANDARD, ans)
    assert A.sameSurface(S)
    return ans

def haken_sum(S1, S2):
    T = S1.triangulation()
    assert S1.locallyCompatible(S2)
    v1, v2 = extract_vector(S1), extract_vector(S2)
    sum_vec = [x1 + x2 for x1, x2 in zip(v1, v2)]
    A = regina.NormalSurface(T, regina.NS_STANDARD, sum_vec)
    assert S1.locallyCompatible(A) and S2.locallyCompatible(A)
    assert S1.eulerChar() + S2.eulerChar() == A.eulerChar()
    return A


def census_lookup(regina_tri):
    """
    Should the input triangulation be in Regina's census, return the
    name of the manifold, dropping the triangulation number.
    """
    hits = regina.Census.lookup(regina_tri)
    hit = hits.first()
    if hit is not None:
        name = hit.name()
        match = re.search('(.*) : #\d+$', name)
        if match:
            return match.group(1)
        else:
            return match

def standard_lookup(regina_tri):
    match = regina.StandardTriangulation.isStandardTriangulation(regina_tri)
    if match:
        return match.manifold()

def closed_isosigs(snappy_manifold, trys=20, max_tets=50):
    """
    Generate a slew of 1-vertex triangulations of a closed manifold
    using SnapPy.
    
    >>> M = snappy.Manifold('m004(1,2)')
    >>> len(closed_isosigs(M, trys=5)) > 0
    True
    """
    M = snappy.Manifold(snappy_manifold)
    assert M.cusp_info('complete?') == [False]
    surgery_descriptions = [M.copy()]

    try:
        for curve in M.dual_curves():
            N = M.drill(curve)
            N.dehn_fill((1,0), 1)
            surgery_descriptions.append(N.filled_triangulation([0]))
    except snappy.SnapPeaFatalError:
        pass

    if len(surgery_descriptions) == 1:
        # Try again, but unfill the cusp first to try to find more
        # dual curves.
        try:
            filling = M.cusp_info(0).filling
            N = M.copy()
            N.dehn_fill((0, 0), 0)
            N.randomize()
            for curve in N.dual_curves():
                D = N.drill(curve)
                D.dehn_fill([filling, (1,0)])
                surgery_descriptions.append(D.filled_triangulation([0]))
        except snappy.SnapPeaFatalError:
            pass

    ans = set()
    for N in surgery_descriptions:
        for i in range(trys):
            T = N.filled_triangulation()
            if T._num_fake_cusps() == 1:
                n = T.num_tetrahedra()
                if n <= max_tets:
                    ans.add((n, T.triangulation_isosig(decorated=False)))
            N.randomize()

    return [iso for n, iso in sorted(ans)]

def best_match(matches):
    """
    Prioritize the most concise description that Regina provides to
    try to avoid things like the Seifert fibered space of a node being
    a solid torus or having several nodes that can be condensed into a
    single Seifert fibered piece.
    """
    
    def score(m):
        if isinstance(m, regina.SFSpace):
            s = 0
        elif isinstance(m, regina.GraphLoop):
            s = 1
        elif isinstance(m, regina.GraphPair):
            s = 2
        elif isinstance(m, regina.GraphTriple):
            s = 3
        elif m is None:
            s = 10000
        else:
            s = 4
        return (s, str(m))
    return min(matches, key=score)

def identify_with_torus_boundary(regina_tri):
    """
    Use the combined power of Regina and SnapPy to try to give a name
    to the input manifold.
    """
    
    kind, name = None, None
    
    P = regina_tri.clone()
    P.finiteToIdeal()
    P.intelligentSimplify()
    M = snappy.Manifold(P.isoSig())
    M.simplify()
    if appears_hyperbolic(M):
        for i in range(100):
            if M.solution_type() == 'all tetrahedra positively oriented':
                break
            M.randomize()
        
        if not M.verify_hyperbolicity(bits_prec=100):
            raise RuntimeError('Cannot prove hyperbolicity for ' +
                               M.triangulation_isosig())
        kind = 'hyperbolic'
        ids = M.identify()
        if ids:
            name = ids[0].name()
    else:
        match = standard_lookup(regina_tri)
        if match is None:
            Q = P.clone()
            Q.idealToFinite()
            Q.intelligentSimplify()
            match = standard_lookup(Q)
        if match is not None:
            kind = match.__class__.__name__
            name = str(match)
        else:
            name = P.isoSig()
    return kind, name
            
    
    

def is_toroidal(regina_tri):
    """
    Checks for essential tori and returns the pieces of the
    associated partial JSJ decomposition.
    
    >>> T = to_regina('hLALAkbccfefgglpkusufk')  # m004(4,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('hvLAQkcdfegfggjwajpmpw')  # m004(0,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('nLLLLMLPQkcdgfihjlmmlkmlhshnrvaqtpsfnf')  # 5_2(10,1)
    >>> T.isHaken()
    True
    >>> is_toroidal(T)[0]
    False

    Note: currently checks all fundamental normal tori; possibly
    the theory lets one just check *vertex* normal tori.
    """
    T = regina_tri
    assert T.isZeroEfficient()
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                ids = [identify_with_torus_boundary(C) for C in pieces]
                return (True, sorted(ids))
                
    return (False, None)


def decompose_along_tori(regina_tri):
    """
    First, finds all essential normal tori in the manifold associated
    with fundamental normal surfaces.  Then takes a maximal disjoint
    collection of these tori, namely the one with the fewest tori
    involved, and cuts the manifold open along it.  It tries to
    identify the pieces, removing any (torus x I) components. 

    Returns: (has essential torus, list of pieces)

    Note: This may fail to be the true JSJ decomposition because there
    could be (torus x I)'s in the list of pieces and it might well be
    possible to amalgamate some of the pieces into a single SFS.
    """
    
    T = regina_tri
    assert T.isZeroEfficient()
    essential_tori = []
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                essential_tori.append(S)

    if len(essential_tori) == 0:
        return False, None
    
    D = nx.Graph()
    for a, A in enumerate(essential_tori):
        for b, B in enumerate(essential_tori):
            if a < b:
                if A.disjoint(B):
                    D.add_edge(a, b)

    cliques = list(nx.find_cliques(D))
    if len(cliques) == 0:
        clique = [0]
    else:
        clique = min(cliques, key=len)
    clique = [essential_tori[c] for c in clique]
    A = clique[0]
    for B in clique[1:]:
        A = haken_sum(A, B)

    X = A.cutAlong()
    X.intelligentSimplify()
    X.splitIntoComponents()
    ids = [identify_with_torus_boundary(C) for C in list(children(X))]
    # Remove products
    ids = [i for i in ids if i[1] not in ('SFS [A: (1,1)]', 'A x S1')]
    return (True, sorted(ids))

def regina_name(closed_snappy_manifold, trys=100):
    """
    >>> regina_name('m004(1,0)')
    'S3'
    >>> regina_name('s006(-2, 1)')
    'SFS [A: (5,1)] / [ 0,-1 | -1,0 ]'
    >>> regina_name('m010(-1, 1)')
    'L(3,1) # RP3'
    >>> regina_name('m022(-1,1)')
    'SFS [S2: (3,2) (3,2) (4,-3)]'
    >>> regina_name('v0004(0, 1)')
    'SFS [S2: (2,1) (4,1) (15,-13)]'
    >>> regina_name('m305(1, 0)')
    'L(3,1) # RP3'
    """
    M = snappy.Manifold(closed_snappy_manifold)
    isosigs = closed_isosigs(M, trys=trys, max_tets=25)
    if len(isosigs) == 0:
        return
    T = to_regina(isosigs[0])
    if T.isIrreducible():
        if T.countTetrahedra() <= 11:
            for i in range(3):
                T.simplifyExhaustive(i)
                name = census_lookup(T)
                if name is not None:
                    return name
            
        matches = [standard_lookup(to_regina(iso)) for iso in isosigs]
        match = best_match(matches)
        if match is not None:
            return str(match)
    else:
        T.connectedSumDecomposition()
        pieces = [regina_name(P) for P in children(T)]
        if None not in pieces:
            return ' # '.join(sorted(pieces))
        
def recognize_mfd(knot):
    """
    Uses regina and snappy to recognize the name of its 0-filling.
    """
    K=snappy.Manifold(knot)
    K_reg=regina_name(K)
    if K_reg is not None:
        return K_reg  
    else:
        try:
            K_reg=decompose_along_tori(to_regina(closed_isosigs(K)[0]))
        except TypeError:
            K_reg=None
        if K_reg is not None and K_reg[0]==True:
            return 'JSJ'+str(K_reg[1])

<>:181: DeprecationWarning: invalid escape sequence \d
<>:181: DeprecationWarning: invalid escape sequence \d
<ipython-input-1-cef6550c8024>:181: DeprecationWarning: invalid escape sequence \d
  match = re.search('(.*) : #\d+$', name)


Next, we load the lists:

In [3]:
low_crossing_knots=[]
for c in range(5,10):
    for K in snappy.HTLinkExteriors(knots_vs_links='knots',crossings=c):
        name=K.identify()[1].name()
        if name[0]=='K':
            name=K.identify()[0].name()
        if name!='5_2':
            low_crossing_knots.append(name)

In [4]:
print(low_crossing_knots)

['5_1', '6_3', '6_2', '6_1', '7_7', '7_6', '7_5', '7_2', '7_3', '7_4', '7_1', '8_14', '8_15', '8_10', '8_8', '8_12', '8_7', '8_13', '8_2', '8_11', '8_6', '8_1', '8_18', '8_5', '8_17', '8_16', '8_9', '8_4', '8_3', '8_20', '8_21', '8_19', '9_30', '9_22', '9_19', '9_25', '9_28', '9_32', '9_24', '9_8', '9_36', '9_15', '9_33', '9_27', '9_31', '9_17', '9_26', '9_23', '9_14', '9_37', '9_20', '9_11', '9_21', '9_12', '9_6', '9_18', '9_16', '9_7', '9_2', '9_34', '9_41', '9_38', '9_29', '9_39', '9_9', '9_13', '9_4', '9_5', '9_40', '9_3', '9_10', '9_35', '9_1', '9_44', '9_45', '9_43', '9_42', '9_46', '9_48', '9_47', '9_49']


In [5]:
len(low_crossing_knots)

81

In [6]:
possible_same_0_surgeries=[] 
for name in low_crossing_knots:
    listname=[]
    try:
        with open(name+'.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                listname.append(row)
            listname=listname[1:]
            low_cros=[]
            high_cros=[]
            for x in listname:
                if int(x[1])<16:
                    low_cros.append(x)
                else:
                    high_cros.append(x)
            listname=[]
            listname.append(low_cros)
            listname.append(high_cros)
            possible_same_0_surgeries.append(listname)
    except FileNotFoundError:
        pass

In [7]:
len(possible_same_0_surgeries)

76

First we order these lists according to the cases that the 0-surgery of the low crossing not is hyperbolic or not.

In [18]:
possible_same_0_surgeries_hyp=[]
possible_same_0_surgeries_non_hyp=[]
for x in possible_same_0_surgeries:
    for name in x[0]:
        K=snappy.Manifold(name[0])
        K.dehn_fill((0,1))
        vol=better_volume(K,try_hard=True)
        if vol==0:
            rec=recognize_mfd(K)
            if rec==None:
                K.randomize()
                rec=recognize_mfd(K)
            print(name[0],' is non-hyperbolic:',rec)
            if x not in possible_same_0_surgeries_non_hyp:
                possible_same_0_surgeries_non_hyp.append(x)
        if vol!=0:
            print(name[0],' is hyperbolic with volume:',vol)
            if x not in possible_same_0_surgeries_hyp:
                possible_same_0_surgeries_hyp.append(x)
    print('---------')

5_1  is non-hyperbolic: SFS [S2: (2,1) (5,2) (10,-9)]
---------
6_3  is hyperbolic with volume: 4.059766425639?
---------
6_2  is hyperbolic with volume: 3.770829451108?
---------
6_1  is non-hyperbolic: SFS [A: (2,1)] / [ 0,1 | 1,-2 ]
9_46  is non-hyperbolic: SFS [A: (2,1)] / [ -1,3 | 1,-2 ]
---------
7_7  is hyperbolic with volume: 6.332666642499?
---------
7_6  is hyperbolic with volume: 6.180274419374?
---------
7_5  is hyperbolic with volume: 5.987810443355?
---------
7_2  is non-hyperbolic: SFS [A: (3,2)] / [ 0,1 | 1,-1 ]
---------
7_3  is hyperbolic with volume: 4.218233644881?
---------
7_4  is non-hyperbolic: JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,3)]')]
9_2  is non-hyperbolic: SFS [A: (4,3)] / [ 0,1 | 1,-1 ]
---------
7_1  is non-hyperbolic: SFS [S2: (2,1) (7,3) (14,-13)]
---------
8_14  is hyperbolic with volume: 8.19064265904?
9_8  is hyperbolic with volume: 7.46191985006?
---------
8_15  is hyperbolic with volume: 9.34081310840?
---------
8_10  is hyper

In [19]:
len(possible_same_0_surgeries_hyp)

64

In [20]:
len(possible_same_0_surgeries_non_hyp)

12

We start with the 12 non-hyperbolic examples. Here we will first remove all hyperbolic 0-fillings with the same Alexander polynomial. In a second step we will use the number of coverings to further distinguish.

In [38]:
start_time = time.time()
possible_same_0_surgeries_both_prop_non_hyp=[]

for x in possible_same_0_surgeries_non_hyp:
    unclear=[]
    for [knot,cros,DT] in x[1]:
        K=snappy.Link(change_notation(DT)).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K)
        if vol==0:
            unclear.append([knot,cros,DT])
    possible_same_0_surgeries_both_prop_non_hyp.append([x[0],unclear]) 
    print('Possible equal surgeries:',[x[0],unclear])
print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

Possible equal surgeries: [[['5_1', '5', '']], [['19nh_000000149', '19', 'lHpMrNiBkQgaODFsJec']]]
Possible equal surgeries: [[['6_1', '6', ''], ['9_46', '9', '']], [['17nh_0000012', '17', 'nmlkjiOqpedcbaGFh'], ['17nh_0000138', '17', 'djFaGnCQlObPheKIM'], ['18nh_00000334', '18', 'nOjMhbRcPefADqKIlG'], ['18nh_00000335', '18', 'gJehqNaMdRoBIPkFcL'], ['18nh_00000400', '18', 'oPjMiNbRecgAFDqKlH'], ['18nh_00000707', '18', 'lHoEMqiBkPgaNDrJfc'], ['19ns_018', '19', 'mQkGONDJrLcHaPEFSiB'], ['19ns_183', '19', 'qKeIaMcpRlBnFjDsgOH'], ['19ns_186', '19', 'qDjFnBlpRcMaIeKsgOH'], ['19nh_000000010', '19', 'ponmlkjQsrfedcbaHGi'], ['19nh_000000156', '19', 'nHqkjOiBmedRgaPFsLc'], ['19nh_000000333', '19', 'pKsIlOqCmGAfRhEbJdN'], ['19nh_000000334', '19', 'fqKrOsPlgEDBhaJIMcn']]]
Possible equal surgeries: [[['7_2', '7', '']], [['16nh_0000443', '16', 'eIGMaCkPOlfnDjBH'], ['17nh_0000009', '17', 'dNjaFIqmElochPkBg']]]
Possible equal surgeries: [[['7_4', '7', ''], ['9_2', '9', '']], [['17nh_0000005', '17', 'nDk

We continue to work with this reduced list. First we try to show again that the remaining manifolds are hyperbolic and if this does not work we try to distinguish using the numbers of branched covers of degree n.

In [41]:
start_time = time.time()
still_unclear=[]
for x in possible_same_0_surgeries_both_prop_non_hyp:
    print(x[0])
    unclear=[]
    for [knot,cros,DT] in x[1]:
        K=snappy.Link(change_notation(DT)).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K,try_hard=True)
        print(knot,vol)
        if vol==0:
            unclear.append([knot,cros,DT])
    if len(unclear)>0:
        still_unclear.append([x[0],unclear])
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

[['5_1', '5', '']]
19nh_000000149 6.47459498758?
----------------
[['6_1', '6', ''], ['9_46', '9', '']]
17nh_0000012 0
17nh_0000138 0
18nh_00000334 0
18nh_00000335 0
18nh_00000400 7.90216370908?
18nh_00000707 0
19ns_018 0
19ns_183 0
19ns_186 0
19nh_000000010 0
19nh_000000156 0
19nh_000000333 0
19nh_000000334 0
----------------
[['7_2', '7', '']]
16nh_0000443 0
17nh_0000009 0
----------------
[['7_4', '7', ''], ['9_2', '9', '']]
17nh_0000005 0
18ns_24 0
18nh_00000598 0
----------------
[['7_1', '7', '']]
18nh_00000002 0
----------------
[['8_1', '8', '']]
----------------
[['8_3', '8', '']]
16ns_03 0
16nh_0000296 0
17ns_06 0
----------------
[['8_20', '8', '']]
16nh_0000011 0
----------------
[['8_19', '8', '']]
16nh_0000001 0
----------------
[['9_5', '9', '']]
16ns_06 0
----------------
[['9_35', '9', '']]
----------------
[['9_1', '9', '']]
----------------
Time taken: 1.325797704855601 minutes 


In [42]:
len(still_unclear)

8

In [43]:
still_unclear

[[[['6_1', '6', ''], ['9_46', '9', '']],
  [['17nh_0000012', '17', 'nmlkjiOqpedcbaGFh'],
   ['17nh_0000138', '17', 'djFaGnCQlObPheKIM'],
   ['18nh_00000334', '18', 'nOjMhbRcPefADqKIlG'],
   ['18nh_00000335', '18', 'gJehqNaMdRoBIPkFcL'],
   ['18nh_00000707', '18', 'lHoEMqiBkPgaNDrJfc'],
   ['19ns_018', '19', 'mQkGONDJrLcHaPEFSiB'],
   ['19ns_183', '19', 'qKeIaMcpRlBnFjDsgOH'],
   ['19ns_186', '19', 'qDjFnBlpRcMaIeKsgOH'],
   ['19nh_000000010', '19', 'ponmlkjQsrfedcbaHGi'],
   ['19nh_000000156', '19', 'nHqkjOiBmedRgaPFsLc'],
   ['19nh_000000333', '19', 'pKsIlOqCmGAfRhEbJdN'],
   ['19nh_000000334', '19', 'fqKrOsPlgEDBhaJIMcn']]],
 [[['7_2', '7', '']],
  [['16nh_0000443', '16', 'eIGMaCkPOlfnDjBH'],
   ['17nh_0000009', '17', 'dNjaFIqmElochPkBg']]],
 [[['7_4', '7', ''], ['9_2', '9', '']],
  [['17nh_0000005', '17', 'nDkFmBjiqgcOeaPLh'],
   ['18ns_24', '18', 'lPjFNMIqKcGaOEDRhB'],
   ['18nh_00000598', '18', 'mHKGrOJLnFCBqPiEad']]],
 [[['7_1', '7', '']], [['18nh_00000002', '18', 'mPLKrJINGFECqO

In [58]:
def subgroups_of_order_n(M,n):
    '''
    Returns the number of subgroups of order n of the fundamental group of M.
    Warning: Works only fast for small n.
    '''
    return len(snappy.Manifold(M).covers(n))

def subgroups_up_to_order_k(M,k):
    '''
    Returns the vector containing the number of subgroups of order n of the fundamental group of M for n=2,...,k.
    Warning: Works only fast for small k.
    '''
    vector=[]
    for n in range(2,k+1):
        vector.append(subgroups_of_order_n(M,n))
    return vector

In [69]:
start_time = time.time()
might_be_the_same=[]
for x in still_unclear:
    print(x[0])
    for y in x[0]:
        unclear=[]
        K=snappy.Manifold(y[0])
        K.dehn_fill((0,1))
        vec0=subgroups_up_to_order_k(K,6)
        print(y[0],'Number of subgroups:',vec0)
        for [knot,cros,DT] in x[1]:
            K=snappy.Link(change_notation(DT)).exterior()
            K.dehn_fill((0,1))
            vec=subgroups_up_to_order_k(K,5)
            print(knot,'Number of subgroups:',vec)
            if vec==vec0[:-1]:
                unclear.append([knot,cros,DT,vec])
        if len(unclear)>1:
            might_be_the_same.append([[y[0],y[1],y[2],vec0],unclear])
                
    
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))
    

[['6_1', '6', '', [1, 2, 1, 3, 3]], ['9_46', '9', '', [1, 5, 4, 6, 24]]]
6_1 Number of subgroups: [1, 2, 1, 3, 3]
17nh_0000012 Number of subgroups: [1, 2, 1, 3]
17nh_0000138 Number of subgroups: [1, 2, 4, 10]
18nh_00000334 Number of subgroups: [1, 5, 4, 6]
18nh_00000335 Number of subgroups: [1, 5, 4, 6]
18nh_00000707 Number of subgroups: [1, 2, 1, 8]
19ns_018 Number of subgroups: [1, 2, 4, 6]
19ns_183 Number of subgroups: [1, 2, 1, 13]
19ns_186 Number of subgroups: [1, 2, 1, 33]
19nh_000000010 Number of subgroups: [1, 2, 1, 3]
19nh_000000156 Number of subgroups: [1, 2, 4, 11]
19nh_000000333 Number of subgroups: [1, 2, 1, 3]
19nh_000000334 Number of subgroups: [1, 2, 1, 3]
9_46 Number of subgroups: [1, 5, 4, 6, 24]
17nh_0000012 Number of subgroups: [1, 2, 1, 3]
17nh_0000138 Number of subgroups: [1, 2, 4, 10]
18nh_00000334 Number of subgroups: [1, 5, 4, 6]
18nh_00000335 Number of subgroups: [1, 5, 4, 6]
18nh_00000707 Number of subgroups: [1, 2, 1, 8]
19ns_018 Number of subgroups: [1, 2, 

In [70]:
might_be_the_same

[[['6_1', '6', '', [1, 2, 1, 3, 3]],
  [['17nh_0000012', '17', 'nmlkjiOqpedcbaGFh', [1, 2, 1, 3]],
   ['19nh_000000010', '19', 'ponmlkjQsrfedcbaHGi', [1, 2, 1, 3]],
   ['19nh_000000333', '19', 'pKsIlOqCmGAfRhEbJdN', [1, 2, 1, 3]],
   ['19nh_000000334', '19', 'fqKrOsPlgEDBhaJIMcn', [1, 2, 1, 3]]]],
 [['9_46', '9', '', [1, 5, 4, 6, 24]],
  [['18nh_00000334', '18', 'nOjMhbRcPefADqKIlG', [1, 5, 4, 6]],
   ['18nh_00000335', '18', 'gJehqNaMdRoBIPkFcL', [1, 5, 4, 6]]]],
 [['8_3', '8', '', [1, 1, 1, 1, 11]],
  [['16ns_03', '16', 'cJoELPiNkBgMDHaF', [1, 1, 1, 1]],
   ['16nh_0000296', '16', 'kGnELhBjOfaMDpIc', [1, 1, 1, 1]]]]]

In [73]:
start_time = time.time()

for x in might_be_the_same:
    print(x[0])
    for [knot,cros,DT,v] in x[1]:
            K=snappy.Link(change_notation(DT)).exterior()
            K.dehn_fill((0,1))
            vec=subgroups_up_to_order_k(K,6)
            print(knot,DT,'Number of subgroups:',vec)
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

['6_1', '6', '', [1, 2, 1, 3, 3]]
17nh_0000012 nmlkjiOqpedcbaGFh Number of subgroups: [1, 2, 1, 3, 3]
19nh_000000010 ponmlkjQsrfedcbaHGi Number of subgroups: [1, 2, 1, 3, 3]
19nh_000000333 pKsIlOqCmGAfRhEbJdN Number of subgroups: [1, 2, 1, 3, 21]
19nh_000000334 fqKrOsPlgEDBhaJIMcn Number of subgroups: [1, 2, 1, 3, 21]
----------------
['9_46', '9', '', [1, 5, 4, 6, 24]]
18nh_00000334 nOjMhbRcPefADqKIlG Number of subgroups: [1, 5, 4, 6, 24]
18nh_00000335 gJehqNaMdRoBIPkFcL Number of subgroups: [1, 5, 4, 6, 24]
----------------
['8_3', '8', '', [1, 1, 1, 1, 11]]
16ns_03 cJoELPiNkBgMDHaF Number of subgroups: [1, 1, 1, 1, 21]
16nh_0000296 kGnELhBjOfaMDpIc Number of subgroups: [1, 1, 1, 1, 21]
----------------
Time taken: 0.028177130222320556 minutes 


In [76]:
K=snappy.Manifold('6_1')
K.dehn_fill((0,1))

In [77]:
subgroups_of_order_n(K,7)

3

In [78]:
K=snappy.Link(change_notation('nmlkjiOqpedcbaGFh')).exterior()
K.dehn_fill((0,1))
subgroups_of_order_n(K,7)

3

In [79]:
K=snappy.Link(change_notation('ponmlkjQsrfedcbaHGi')).exterior()
K.dehn_fill((0,1))
subgroups_of_order_n(K,7)

3

In [80]:
K=snappy.Manifold('9_46')
K.dehn_fill((0,1))
subgroups_of_order_n(K,7)

9

In [81]:
K=snappy.Link(change_notation('nOjMhbRcPefADqKIlG')).exterior()
K.dehn_fill((0,1))
subgroups_of_order_n(K,7)

9

In [82]:
K=snappy.Link(change_notation('gJehqNaMdRoBIPkFcL')).exterior()
K.dehn_fill((0,1))
subgroups_of_order_n(K,7)

9

For the remaining examples this seems not to help. So here we use regina to recognize the manifold and check if we get a match.

We recall that:

6_1 (0,1) is non-hyperbolic: SFS [A: (2,1)] / [ 0,1 | 1,-2 ]

9_46 (0,1) is non-hyperbolic: SFS [A: (2,1)] / [ -1,3 | 1,-2 ]


In [83]:
K=snappy.Link(change_notation('nmlkjiOqpedcbaGFh')).exterior()
K.dehn_fill((0,1))
recognize_mfd(K)

'SFS [A: (2,1)] / [ 2,11 | 1,5 ]'

In [84]:
K=snappy.Link(change_notation('ponmlkjQsrfedcbaHGi')).exterior()
K.dehn_fill((0,1))
recognize_mfd(K)

'SFS [A: (2,1)] / [ 2,13 | 1,6 ]'

In [85]:
K=snappy.Link(change_notation('nOjMhbRcPefADqKIlG')).exterior()
K.dehn_fill((0,1))
recognize_mfd(K)

"JSJ[('hyperbolic', 's785')]"

In [86]:
K=snappy.Link(change_notation('gJehqNaMdRoBIPkFcL')).exterior()
K.dehn_fill((0,1))
recognize_mfd(K)

"JSJ[('hyperbolic', 's780')]"

The Seifert fibered space over the annulus we can distinguish via the traces of their gluing maps. The other manifolds are right away not diffoemorphic since they have different JSJ decompositions. This finishes the classification in the non-hyperbolic case.

We continue with the hyperbolic 0-fillings. Here we use the verified volumes to distinguish. First we create lists of manifolds where the volume overlaps. If the volume cannot be comuted we try to verify the manifold to be non-hyperbolic.

In [90]:
start_time = time.time()
pairs_with_same_or_unclear_volume=[]
for x in possible_same_0_surgeries_hyp:
    for y in x[0]:
        unclear=[]
        K=snappy.Manifold(y[0])
        K.dehn_fill((0,1))
        vol0=better_volume(K,try_hard=True)
        print(y[0],'Volume:',vol0)
        for [knot,cros,DT] in x[1]:
            K=snappy.Link(change_notation(DT)).exterior()
            K.dehn_fill((0,1))
            vol=better_volume(K,try_hard=True)
            if vol==0:
                name=recognize_mfd(K)
                if name is None:
                    print('We could not determine the hyperbolicity of:',knot,DT)
                    unclear.append([knot,cros,DT])
            if vol!=0:
                if vol.overlaps(vol0):
                    unclear.append([knot,cros,DT])
        if len(unclear)>1:
            pairs_with_same_or_unclear_volume.append([[y[0],y[1],y[2]],unclear])
                
    
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

6_3 Volume: 4.059766425639?
----------------
6_2 Volume: 3.770829451108?
----------------
7_7 Volume: 6.332666642499?
----------------
7_6 Volume: 6.180274419374?
----------------
7_5 Volume: 5.987810443355?
----------------
7_3 Volume: 4.218233644881?
----------------
8_14 Volume: 8.19064265904?
9_8 Volume: 7.46191985006?
----------------
8_15 Volume: 9.34081310840?
----------------
8_10 Volume: 7.790015973497?
----------------
8_8 Volume: 6.38487304951?
----------------
8_12 Volume: 7.64659248195?
----------------
8_7 Volume: 6.111659915355?
----------------
8_13 Volume: 7.259889578204?
----------------
8_2 Volume: 4.703642059133?
----------------
8_11 Volume: 7.186378673021?
----------------
8_6 Volume: 6.33266664250?
----------------
8_18 Volume: 11.14721822567?
9_24 Volume: 9.55789756113?
----------------
8_5 Volume: 6.73630906712?
----------------
8_17 Volume: 9.65085003623?
----------------
8_16 Volume: 9.78375114088?
----------------
8_9 Volume: 5.65624417667?
----------------


In [91]:
pairs_with_same_or_unclear_volume

[]

This proves that all other 0-fillings of manifolds with the same Alexander polynomial are hyperbolic with different volume. This is somehow expected since these manifolds will usually be hyperbolic with larger volume.

This proves that there is no pair of prime knots (K,K') such that K has at most 9 corrsings and K' has between (including) 16 and 19 crossings with K(0,1)=K'(0,1).